In [ ]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import numpy as np
import math 

Check out the Data

In [ ]:
df = pd.read_csv("Portugal 2022-08-01 to 2023-08-01.csv", na_filter = True)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
df.nunique()

Exploratory Data Analysis

1.Check for Duplicates

In [ ]:
# ver se existem linhas duplicadas se sim apager se não não fazer nada
print ( df.duplicated().sum() )

Do nothing

2.Check for missing Data

In [ ]:
# ver se existem celulas vazias e se existirem temos de decidir preencher ou apagalas
sns.heatmap(df.isnull(),yticklabels= False,  cbar = False, cmap = 'viridis')

Analise the atribute to decide what to do

In [ ]:
# analisar a coluna preciptype e tentar chegar a uma decisão em o que fazer
print(df['preciptype'])
print('\n')
print ( df['preciptype'].describe() )
print('\n')
print ( df['preciptype'].value_counts() )

Desicion: Delete the colum was our decision because there were too manny black spaces

In [ ]:
# Decidimos como temos muitas entradas em branco que devemos apagar a coluna

df.drop(['preciptype'], inplace = True, axis = 1) # apaga-mos a coluna 'preciptype'

Check there is no more missing data

In [ ]:
# ver que não existem mais colunas vazias
sns.heatmap(df.isnull(),yticklabels= False,  cbar = False, cmap = 'viridis')

3.Check for unique Data

In [ ]:
df.nunique()

Realise that the atributes: snow, snowdepth and name are all unique data

In [ ]:
# verificar que na colunas name, snow e snowdepth temos apenas um valor para apagar
print ( df['name'].nunique() )
print("-------------------------------------------------------------------------------------------")
print ( df['snow'].nunique() )
print("-------------------------------------------------------------------------------------------")
print ( df['snowdepth'].nunique())

Alasise if we can extract data from them

In [ ]:
print ( df['name'].describe() )# Concluimos que temos apenas um nome ou seja podemos apagar esta coluna pois não trará nada de novo para o modelo
print("-------------------------------------------------------------------------------------------")
print ( df['snow'].describe() )# Concluios que nunca nevou logo esta coluna é unutil
print("-------------------------------------------------------------------------------------------")
print ( df['snowdepth'].describe())# Como nunca nevou nunca tivemos profundidade da neve logo esta coluna é unutil

Realise that we can not and procide to there delicion

In [ ]:
df.drop(['name'], inplace = True, axis = 1) # apaga-mos a coluna 'name'
df.drop(['snow'], inplace = True, axis = 1) # apaga-mos a coluna 'snow'
df.drop(['snowdepth'], inplace = True, axis = 1) # apaga-mos a coluna 'snowdepth'

Verificie that the atributes are gone now

In [ ]:
df.head()

4.Check for redundant data

Realise that the atributes: sunrise, sunset e datetime give repeated Data

In [ ]:
# Analise do atributo sunrise

print ( df['sunrise'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['sunrise'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['sunrise'])

In [ ]:
# Analise do atributo sunset

print ( df['sunset'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['sunset'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['sunset'])

In [ ]:
# Analise do atributo datetime

print ( df['datetime'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['datetime'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['datetime'])

Procide to the extracion of the all the data that those atributes gave us and covert it all to new atributes

In [ ]:
#Transformar os atributos em datetime
df['sunrise'] = pd.to_datetime(df['sunrise'])
df['sunset'] = pd.to_datetime(df['sunset'])

# Criar colunas para dia, mês, ano
df['Year']   = df['sunrise'].dt.year
df['Day']    = df['sunrise'].dt.day
df['Month']  = df['sunrise'].dt.month

# Criar colunas para  a hora de o sol nascer,o minuto de o sol nascer e o segundo de o sol nascer
df['hour_rise']   = df['sunrise'].dt.hour
df['minute_rise'] = df['sunrise'].dt.minute
df['second_rise'] = df['sunrise'].dt.second

# Criar colunas para a hora de sol se pôr, o minuto de o sol se pôr e o segundo de o sol se pôr
df['hour_set']   = df['sunset'].dt.hour
df['minute_set'] = df['sunset'].dt.minute
df['second_set'] = df['sunset'].dt.second

Delete the old atributes because they do not have any Data that we do not have now

In [ ]:
df.drop(['sunrise'], inplace = True, axis = 1) #  apaga-mos a coluna 'sunrise'  pois já temos essa informação
df.drop(['sunset'], inplace = True, axis = 1) #   apaga-mos a coluna 'sunset'   pois já temos essa informação
df.drop(['datetime'], inplace = True, axis = 1) # apaga-mos a coluna 'datetime' pois já temos essa informação

Realise that the atributes: conditions, description e icon give us the same Data but in diferent forms

In [ ]:
# analise do atributo conditions
print ( df['conditions'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['conditions'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['conditions'])

In [ ]:
# analise do atributo description
print ( df['description'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['description'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['description'])

In [ ]:
# analise do atributo icon
print ( df['icon'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['icon'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['icon'])

Gruop all of this atributs and realise that the description atribut has more data

In [ ]:
# Criar um novo dataset com apenas os três atributos em baixo
iguais = ['conditions', 'icon','description']
df_novo = df[iguais]

# Calcular a contagem de todas as combinações de categorias de cada coluna
contagem_combinacoes = df.groupby(['conditions', 'icon','description']).size().reset_index(name='count')

# exibir um dataset com a contagem da intercessão das diferentes combinações de categorias
print(contagem_combinacoes)

Creat new binary atributs through the old catgories in description

In [ ]:
# Criar os atributos binários
df['rain_morning'] = (df['description'].str.contains('morning rain') |
                      df['description'].str.contains('Partly cloudy throughout the day with rain in the morning and afternoon') |
                      df['description'].str.contains('Partly cloudy throughout the day with a chance of rain throughout the day') |
                      df['description'].str.contains('with rain clearing later') |
                      df['description'].str.contains('with rain')).astype(int)

In [ ]:
df['rain_afternoon'] = (df['description'].str.contains('afternoon rain') |
                        df['description'].str.contains('rain in the morning and afternoon') |
                        df['description'].str.contains('Partly cloudy throughout the day with rain')
                       ).astype(int)

In [ ]:
df['cloud_morning'] = (df['description'].str.contains('Clearing in the afternoon') |
                       df['description'].str.contains('Partly cloudy throughout the day') |
                       df['description'].str.contains('Cloudy skies throughout the day')
                      ).astype(int)

In [ ]:
df['cloud_afternoon'] = (df['description'].str.contains('Partly cloudy throughout the day') |
                         df['description'].str.contains('Becoming cloudy in the afternoon') |
                         df['description'].str.contains('Cloudy skies throughout the day')
                        ).astype(int)

In [ ]:
df['clear_morning'] = (df['description'].str.contains('Clear conditions throughout the day.') |
                      df['description'].str.contains('Becoming cloudy in the afternoon.')
                     ).astype(int)

In [ ]:
df['clear_afternoon'] = (df['description'].str.contains('Clear conditions throughout the day') |
                        df['description'].str.contains('Clearing in the afternoon')
                       ).astype(int)

Delete the old atributes because they do not have any Data that we do not have now

In [ ]:
df.drop(['conditions'], inplace = True, axis = 1)  # apaga-mos a coluna 'conditions'  pois já temos essa informação
df.drop(['description'], inplace = True, axis = 1) # apaga-mos a coluna 'description' pois já temos essa informação
df.drop(['icon'], inplace = True, axis = 1)        # apaga-mos a coluna 'icon'        pois já temos essa informação

Realise that the atributes: precipprob and (rain_morning and rain_afternoon) give us the same Data but in diferent forms

In [ ]:
# analise do atributo precipprob
print ( df['precipprob'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['precipprob'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['precipprob'])
# vemos que é um atributo que é ou 0 ou 100, para dizer se chove ou não logo como já temos as 
#colunas rain_morning and rain_afternoon concluimos que ele é inutil

Delete the atribute precipprob

In [ ]:
df.drop(['precipprob'], inplace = True, axis = 1) # apaga-mos a coluna 'precipprob'  pois já temos essa informação

Verificie all of our changes

In [ ]:
df.head()

5.Check if you can't get more data

Realise that we can extract more relevant data from the atribute winddir 

In [ ]:
# analise do atributo winddir
print ( df['winddir'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['winddir'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['winddir'].nunique())
print("-------------------------------------------------------------------------------------------")
print(df['winddir'])
# este é o atributo que nos dá a direção do vento em float

This is the atribute that give us the direcion of the wind on a float


This atribute was manny unique values, so whoud me better if we had the catgoric direcions: north, northeast, north-northeast, east-northeast, east, east-southeast, southeast, south, south-southeast, southwest, south-southwest, west-southwest, west, west-northwest, northwest, north-northwest, north-northeast

In [ ]:
#função que categorira a direção do vento
def categorize_wind_direction(wind_direction):
    
    categories = ["north", "northeast","north-northeast","east-northeast","east","east-southeast",
                  "southeast", "south","south-southeast", "southwest", "south-southwest" ,"west-southwest",
                  "west", "west-northwest" ,"northwest","north-northwest", "north-northeast"]
    
    index = round( (wind_direction % 360) / 22.5) % 16
    return categories[index]

# Aplicar a função à coluna 'winddir'
df['winddir'] = df['winddir'].apply(categorize_wind_direction)

# categorizar o atributo
df['winddir'] = df['winddir'].astype('category')

Now we shoud extract all of this catgories and creat new binary atributes

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()

lb_results = lb.fit_transform(df['winddir'])

df_lb_results = pd.DataFrame(lb_results,columns = lb.classes_)

df = pd.concat([df, df_lb_results], axis = 1)

# estamos a criar os novos atributos binários consuante as categorias de winddir

Delete the atribute winddir

In [ ]:
df.drop(['winddir'], inplace = True, axis = 1) # apaga-mos a coluna 'winddir'  pois já temos essa informação

Realise that we can extract more relevant data from the atribute stacions

In [ ]:
# analise do atributo stacions
print ( df['stations'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['stations'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['stations'])

# este é um atributo que nos dá qual foi a estação meteriológica que registou os valores

Let as creat new atributes from the old atribute stacions

In [ ]:
# Criar uma lista única de todas as estações
all_stations = set(','.join(df['stations']).split(','))

# Criar colunas binárias para cada estação
for station in all_stations:
    df[station] = df['stations'].apply(lambda x: int(station in x))

Delete the the atribute stations

In [ ]:
df = df.drop('stations', axis=1)# apaga-mos a coluna 'stations'  pois já temos essa informação

Verifie that the new atributs 08534099999, 08536099999, LPPT and only have one value

In [ ]:
# analise do atributo 08534099999
print ( df['08534099999'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['08534099999'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['08534099999'].nunique())

In [ ]:
# analise do atributo 08536099999
print ( df['08536099999'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['08536099999'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['08536099999'].nunique())

In [ ]:
# analise do atributo LPPT
print ( df['LPPT'].describe() )
print("-------------------------------------------------------------------------------------------")
print ( df['LPPT'].value_counts() )
print("-------------------------------------------------------------------------------------------")
print(df['LPPT'].nunique())

Drop all of the atributes whith one value

In [ ]:
df = df.drop('08534099999', axis=1)# apaga-mos a coluna '08534099999'  pois já temos essa informação
df = df.drop('08536099999', axis=1)# apaga-mos a coluna '08536099999'  pois já temos essa informação
df = df.drop('LPPT', axis=1)       # apaga-mos a coluna 'LPPT'         pois já temos essa informação

Verificie all of our changes

In [ ]:
df.head()

6.Check if we shoud bind the atributes

In [ ]:
df.unique()

Realise that we shoud bin humidity and cloudcover beacause of the large unique values quantity

In [ ]:
# analise do atributo humidity
print ( df['humidity'].describe() )
print("-------------------------------------------------------------------------------------------")
print(df['humidity'].value_counts())
print("-------------------------------------------------------------------------------------------")
print(df['humidity'].nunique())

In [ ]:
# analise do atributo cloudcover
print ( df['cloudcover'].describe() )
print("-------------------------------------------------------------------------------------------")
print(df['cloudcover'].value_counts())
print("-------------------------------------------------------------------------------------------")
print(df['cloudcover'].nunique())

Bind the atribute humidity

In [ ]:
# Função para fazer o binning
def bin_humidity(value):
    decimal_part = value - int(value)
    
    if decimal_part <= 0.2:
        return math.floor(value)
    elif 0.3 <= decimal_part <= 0.6:
        return int(value) + 0.5
    else:
        return math.ceil(value)

# Aplicar a função à coluna 'humidity'
df['humidity'] = df['humidity'].apply(bin_solar_energy)

# Categorizar os valores numéricos
df['humidity'] = df['humidity'].astype('category')

Bind the atribute cloudcover

In [ ]:
# Função para fazer o binning
def bin_cloudcover(value):
    decimal_part = value - int(value)
    
    if decimal_part <= 0.2:
        return math.floor(value)
    elif 0.3 <= decimal_part <= 0.6:
        return int(value) + 0.5
    else:
        return math.ceil(value)

# Aplicar a função à coluna 'cloudcover'
df['cloudcover'] = df['cloudcover'].apply(bin_solar_energy)

# Categorizar os valores numéricos
df['cloudcover'] = df['cloudcover'].astype('category')

Verificie all of our changes

In [ ]:
df.head()

7.Check the correlation of the atributes

In [ ]:
corr_matrix = df.corr()
f,ax =  plt.subplots(figsize = (100,100))
sns.heatmap(corr_matrix, vmin=-1, vmax=1, annot=True, square=True)

Realise that the correlation of the atributes: feelslikemin, feelslikemax and feelslike have a hight correlacion betten them and temp

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'feelslikemax', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'feelslike', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'feelslikemin', data=df, kind='scatter')

Realise that the correlation of the atributes: feelslikemin, feelslikemax and feelslike have a hight correlacion betten them and tempmax

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmax', y= 'feelslikemax', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmax', y= 'feelslikemin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmax', y= 'feelslike', data=df, kind='scatter')

Realise that the correlation of the atributes: feelslikemin, feelslikemax and feelslike have a hight correlacion betten them and tempmin

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmin', y= 'feelslikemax', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmin', y= 'feelslikemin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'tempmin', y= 'feelslike', data=df, kind='scatter')

By observing the graphs and the confusion matrix, we concluded that the attributes of feels like and temperature are almost maximum, so we proceeded to eliminate them.

In [ ]:
df.drop(['feelslikemin'], inplace = True, axis = 1)
df.drop(['feelslikemax'], inplace = True, axis = 1)
df.drop(['feelslike'], inplace = True, axis = 1)

#Por observasão dos gráficos e da matriz de confusão concluimos que os atributos de feels like 
# e a temperatura são quase maximos logo procedemos á sua eleminação

Realise that the correlation of the atribute temp have a hight correlacion betten them tempmin and tempmax

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'tempmin', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'temp', y= 'tempmax', data=df, kind='scatter')

For the same reason as before we shall delete the atribut temp

In [ ]:
df.drop(['temp'], inplace = True, axis = 1)

Realise that the correlation of the atributes solarradiation and uvindex have a hight correlacion betten them solarenergy

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'solarradiation', y= 'solarenergy', data=df, kind='scatter')

In [ ]:
sns.set_style("whitegrid")
sns.jointplot(x= 'uvindex', y= 'solarenergy', data=df, kind='scatter')

Once again for the same reasons we will eliminate the solarradiation uvindex and attributes

In [ ]:
df.drop(['solarradiation'], inplace = True, axis = 1)
df.drop(['uvindex'], inplace = True, axis = 1)

By observing the correlation matrix we can see that clear_morning and cloud_morning have a correlation factor of -1, that is, when one goes up the other goes down, that is, we can remove one of them. The same reasoning can be said for the clear_afternoon and cloud_afternoon attributes.

In [ ]:
df.drop(['clear_morning'], inplace = True, axis = 1)
df.drop(['clear_afternoon'], inplace = True, axis = 1)

#Pela observação da matriz de correlação conseguimos observar que o clear_morning e cloud_morning 
#têm um fator de correlação de -1 ou seja quando um sobe o outro desce ou seja podemos retirar um deles o mesmo racioncinio 
#pode ser dito para os atributos clear_afternoon e cloud_afternoon

Verificie all of our changes

In [ ]:
df.head()

Let as check the the new correlacion matrix

In [ ]:
corr_matrix = df.corr()
f,ax =  plt.subplots(figsize = (100,100))
sns.heatmap(corr_matrix, vmin=-1, vmax=1, annot=True, square=True)

Normalise every single thing to prevent diferent oder number interfiring whith our model